In [9]:
import numpy as np
from itertools import product,permutations
import galois



In [10]:
H = np.array([[1, 0, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 1, 0], [0, 1, 1, 1, 0, 0, 1]])

G = np.array(
    [
        [1, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 1, 1, 1, 1],
    ]
)

n = 7
k = 4
r = k / n

In [11]:
# Generate all 4-bit patterns
patterns = list(product([0, 1], repeat=4))

codebook = []
information_patterns=[]

# Print the patterns
for pattern in patterns:
    codeword = pattern @ G % 2
    print(pattern, "is mapped onto", codeword)
    information_patterns.append(np.array(pattern))
    codebook.append(codeword)

information_patterns=np.array(information_patterns)

information_patterns=information_patterns[np.argsort(np.sum(information_patterns,axis=1))]

# print(information_patterns)


(0, 0, 0, 0) is mapped onto [0 0 0 0 0 0 0]
(0, 0, 0, 1) is mapped onto [0 0 0 1 1 1 1]
(0, 0, 1, 0) is mapped onto [0 0 1 0 1 0 1]
(0, 0, 1, 1) is mapped onto [0 0 1 1 0 1 0]
(0, 1, 0, 0) is mapped onto [0 1 0 0 0 1 1]
(0, 1, 0, 1) is mapped onto [0 1 0 1 1 0 0]
(0, 1, 1, 0) is mapped onto [0 1 1 0 1 1 0]
(0, 1, 1, 1) is mapped onto [0 1 1 1 0 0 1]
(1, 0, 0, 0) is mapped onto [1 0 0 0 1 1 0]
(1, 0, 0, 1) is mapped onto [1 0 0 1 0 0 1]
(1, 0, 1, 0) is mapped onto [1 0 1 0 0 1 1]
(1, 0, 1, 1) is mapped onto [1 0 1 1 1 0 0]
(1, 1, 0, 0) is mapped onto [1 1 0 0 1 0 1]
(1, 1, 0, 1) is mapped onto [1 1 0 1 0 1 0]
(1, 1, 1, 0) is mapped onto [1 1 1 0 0 0 0]
(1, 1, 1, 1) is mapped onto [1 1 1 1 1 1 1]


In [12]:
def ML_decoder(codebook, received_pattern):
    # Checks correlation of all codewords with received pattern
    min_weight = 0
    chosen_codeword_idx = -1
    for i, codeword in enumerate(codebook):
        test_weight = np.sum((1 - 2 * codeword) * received_pattern)  # bpsk
        if test_weight > min_weight:
            chosen_codeword_idx = i
            min_weight = test_weight

    return codebook[chosen_codeword_idx], len(codebook)

In [13]:
def gaussian_elimination(matrix):
    # Determine reduced echelon form
    rows, cols = matrix.shape

    lead = 0
    for c in range(cols):
        if lead >= rows:
            break

        for r in range(lead, rows):
            if matrix[r, c] != 0:
                matrix[[lead, r]] = matrix[[r, lead]]  # pivot found swap row to the top
                for e in range(rows):
                    if e != lead and matrix[e, c] != 0:
                        matrix[e, :] = (matrix[e, :] + matrix[lead, :]) % 2

                lead = lead + 1

    return matrix




In [14]:
def OSD_decoder(generator_matrix, received_pattern, order):
    k, n = generator_matrix.shape
    gf2 = galois.GF2

    absolute_received_values = np.abs(received_pattern)
    permutation_MR_basis = np.flip(
        np.argsort(absolute_received_values)
    )  # order indices according to descending order

    while np.linalg.matrix_rank(gf2(G[:, permutation_MR_basis])) != k:
        # print("hello")
        break

    # print(permutation_MR_basis)

    # print(G)
    # print(G[:, permutation_MR_basis])

    reduced_echelon_form = gaussian_elimination(G[:, permutation_MR_basis])
    position_unit_vectors = []
    dependent_col = []
    lead = 0
    for col in range(n):
        if lead < k and reduced_echelon_form[lead, col] == 1:
            position_unit_vectors.append(col)
            lead = lead + 1
        else:
            dependent_col.append(col)

    permutation_to_MRI = np.array(position_unit_vectors + dependent_col)

    G_mri = reduced_echelon_form[:, permutation_MR_basis]

    full_permutation = np.zeros(n, dtype=int)
    for i in range(n):
        full_permutation[i] = permutation_MR_basis[permutation_to_MRI[i]]

    full_permuted_received_values = received_pattern[full_permutation]

    min_weight = 0
    checked_codeword_counter = 0
    permuted_estimate=np.zeros(n,dtype=int)

    while (np.sum(information_patterns[checked_codeword_counter,:]) <= order):
        # print(information_patterns[checked_codeword_counter])
        permuted_codeword_candidate = (
            information_patterns[checked_codeword_counter] @ G_mri
        ) % 2

        test_weight = np.sum(
            (1 - 2 * permuted_codeword_candidate) * full_permuted_received_values
        )

        # print(permuted_codeword_candidate,'with weight ',test_weight)
        if test_weight > min_weight:
            permuted_estimate = permuted_codeword_candidate
            min_weight = test_weight
        checked_codeword_counter=checked_codeword_counter+1
    
    #undo all permutations
    estimated_codeword=np.zeros(n,dtype=int)
    estimated_codeword[full_permutation]=permuted_estimate

    return estimated_codeword,checked_codeword_counter


In [15]:
test = np.array([1, 1.1, 1.3, 1.4, 1.5, 1.6, -0.1])

a,b=OSD_decoder(G, test, 1)

print("estimate:",a)

print("trials:",b)

a,b=ML_decoder(codebook,test)

print("estimate:",a)

print("trials:",b)


estimate: [0 0 0 0 0 0 0]
trials: 5
estimate: [0 0 0 0 0 0 0]
trials: 16


In [16]:
eb_n0_list_db = [2, 2.5, 3]

fer_osd = []
fer_ml = []


for eb_n0_db in eb_n0_list_db:
    snr = r * 10**(eb_n0_db/10)
    sigma = 1 / np.sqrt(2 * snr)
    for numsim in range(5):
        codeword = codebook[np.random.randint(2**k)]
        modulated = 1 - 2 * codeword

        # AWGN channel
        received = modulated + sigma*np.random.randn(n)
        a,b=OSD_decoder(G, received, 2)

        print("codeword:",codeword)
        print("estimate:",a)

        print("trials:",b)

        a,b=ML_decoder(codebook,received)

        

        print("estimate:",a)

        print("trials:",b)

codeword: [1 0 0 0 1 1 0]
estimate: [0 0 1 0 1 1 0]
trials: 11
estimate: [1 0 0 0 1 1 0]
trials: 16
codeword: [0 1 1 1 0 0 1]
estimate: [0 1 1 0 0 0 1]
trials: 11
estimate: [0 1 1 1 0 0 1]
trials: 16
codeword: [1 1 0 0 1 0 1]
estimate: [1 1 0 0 0 0 1]
trials: 11
estimate: [1 1 0 0 1 0 1]
trials: 16
codeword: [1 1 0 0 1 0 1]
estimate: [1 1 0 0 0 1 1]
trials: 11
estimate: [1 1 0 0 1 0 1]
trials: 16
codeword: [1 0 0 1 0 0 1]
estimate: [1 0 0 0 1 0 1]
trials: 11
estimate: [1 0 0 1 0 0 1]
trials: 16
codeword: [0 1 0 1 1 0 0]
estimate: [0 0 0 1 1 1 0]
trials: 11
estimate: [0 1 0 1 1 0 0]
trials: 16
codeword: [0 0 1 0 1 0 1]
estimate: [0 0 1 0 1 1 1]
trials: 11
estimate: [0 0 1 0 1 0 1]
trials: 16
codeword: [1 1 0 0 1 0 1]
estimate: [1 1 0 1 1 0 0]
trials: 11
estimate: [1 1 0 0 1 0 1]
trials: 16
codeword: [0 1 1 1 0 0 1]
estimate: [0 0 1 1 0 1 1]
trials: 11
estimate: [0 1 1 1 0 0 1]
trials: 16
codeword: [1 0 0 0 1 1 0]
estimate: [1 0 0 0 1 1 1]
trials: 11
estimate: [1 0 0 0 1 1 0]
trials: 16
